* A = state transition probabilities
* B = observation probability matrix
* N = number of states in the model
* M = number of observation symbols
* T = length of the observation sequence

ref : [hmm forward and backward](https://www.programmerall.com/article/1357868031/)

In [ ]:
import numpy as np

#### HMM forward algorithm
given hmm model $\lambda = (A, B, \pi)$
return forward prob $\alpha_t(i)$ and observation sequence prob $P(O|\lambda)$

In [21]:
def hmm_forward(A, B, pi, O):
    # define
    T = len(O)
    N = len(A[0])
    alpha = np.zeros((N, T))
    # 1. initialisation: calculate the first col of alpha
    for i in range(N):
        alpha[i][0] = pi[i]*B[i][O[0]]

    # 2. iteration
    for t in range(1, T):
        for i in range(N):
            a = 0
            for j in range(N):
                a += alpha[j][t-1]*A[j][i]
            alpha[i][t] = a*B[i][O[t]]

    # 3. observation sequence prob
    proba = 0
    for i in range(N):
        proba += alpha[i][-1]
        print(proba)
    return proba,alpha


A = [[0.5,0.2,0.3],[0.3,0.5,0.2],[0.2,0.3,0.5]]
B = [[0.5,0.5],[0.4,0.6],[0.7,0.3]]
pi = [0.2,0.4,0.4]
O = [0,1,0,1]

p, a = hmm_forward(A,B,pi,O)
print(p, a)

0.021077899999999997
0.046266379999999996
0.06009079999999999
0.06009079999999999 [[0.1        0.077      0.04187    0.0210779 ]
 [0.16       0.1104     0.035512   0.02518848]
 [0.28       0.0606     0.052836   0.01382442]]


#### HMM backward algorithm
given hmm model $\lambda = (A, B, \pi)$
return forward prob $\beta_t(i)$ and observation sequence prob $P(O|\lambda)$

In [35]:
def hmm_backward(A, B, pi, O):
    # define
    T = len(O)
    N = len(A[0])
    beta = np.zeros((N, T))

    # 1. initialisation: calculate the first col of alpha
    for i in range(N):
        beta[i][-1] = 1


    # backward iteration
    for t in reversed(range(T-1)):
        for i in range(N):
            for j in range(N):
                beta[i][t] += A[i][j] * B[j][O[t+1]] * beta[j][t+1]

    # 3. observation sequence prob
    proba = 0
    for i in range(N):
        proba += pi[i]*B[i][O[0]]*beta[i][0]

    return proba,beta

A = [[0.5,0.2,0.3],[0.3,0.5,0.2],[0.2,0.3,0.5]]
B = [[0.5,0.5],[0.4,0.6],[0.7,0.3]]
pi = [0.2,0.4,0.4]
O = [0,1,0,1]

p, b = hmm_backward(A,B,pi,O)
print(p, b)

0.06009079999999999 [[0.112462 0.2461   0.46     1.      ]
 [0.121737 0.2312   0.51     1.      ]
 [0.104881 0.2577   0.43     1.      ]]


#### Baum-Welch algorithm (EM algorithm)
given {$(O_1, I_1)$, $(O_2, I_2)$, ... $(O_s, I_s)$} observation and its corresponding status sequence
return estimated HMM parameters